In [11]:
from ortools.sat.python import cp_model

In [12]:
tablero_m = [
    [3, 5, 4, 2, 1, 6],
    [3, 2, 5, 6, 4, 1],
    [2, 4, 3, 1, 5, 6],
    [5, 6, 4, 2, 3, 1],
    [2, 5, 3, 6, 4, 1],
    [1, 3, 4, 5, 6, 2]
]

tablero_w = [
    [2, 4, 5, 3, 6, 1],
    [3, 5, 4, 2, 1, 6],
    [1, 3, 6, 2, 4, 5],
    [3, 2, 5, 6, 4 ,1],
    [6, 4, 2, 1, 3, 5],
    [6, 4, 3, 1, 5, 2]
]


In [13]:
model = cp_model.CpModel()

parejas = [[model.NewBoolVar(f"parejas_{m}_{w}") for w in range(6)] for m in range(6)]


n = 6

for m in range(6):
    model.Add(sum(parejas[m][w] for w in range(6)) == 1)
for w in range(6):
    model.Add(sum(parejas[m][w] for m in range(6)) == 1)


minRank = 4
for w in range(6):
    for m in range(6):
        if tablero_w[w][m] > minRank:
            model.Add(parejas[m][w] == 0) 

for m in range(6):
    for w in range(6):
        for m0 in range(6):
            for w0 in range(6):
                if tablero_m[m][w0] < tablero_m[m][w] and tablero_w[w0][m] < tablero_w[w0][m0]:
                    model.Add(parejas[m][w] + parejas[m0][w0] <= 1)

In [14]:
class SolutionPrinter(cp_model.CpSolverSolutionCallback):
    def __init__(self, parejas):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self._parejas = parejas
        self.count = 0
        self.resultados = []

    def OnSolutionCallback(self):
        self.count += 1
        matching = []
        for m in range(6):
            for w in range(6):
                if self.Value(self._parejas[m][w]) == 1:
                    matching.append((m, w))
                    break
        self.resultados.append(matching)
       
        print(f"Solución {self.count}:")
        for m, w in matching:
            print(f"  Hombre {m+1}  —  Mujer {w+1}   (rank hombre: {tablero_m[m][w]}, rank mujer: {tablero_w[w][m]})")
        print("")

    def get_results(self):
        return self.resultados
    

In [15]:
solver = cp_model.CpSolver()


printer = SolutionPrinter(parejas)

solver.SearchForAllSolutions(model, printer)

print(f"Total soluciones encontradas: {printer.count}")
if printer.count == 0:
    print("No existe solucion.")

Solución 1:
  Hombre 1  —  Mujer 4   (rank hombre: 2, rank mujer: 3)
  Hombre 2  —  Mujer 5   (rank hombre: 4, rank mujer: 4)
  Hombre 3  —  Mujer 2   (rank hombre: 4, rank mujer: 4)
  Hombre 4  —  Mujer 6   (rank hombre: 1, rank mujer: 1)
  Hombre 5  —  Mujer 3   (rank hombre: 3, rank mujer: 4)
  Hombre 6  —  Mujer 1   (rank hombre: 1, rank mujer: 1)

Solución 2:
  Hombre 1  —  Mujer 4   (rank hombre: 2, rank mujer: 3)
  Hombre 2  —  Mujer 3   (rank hombre: 5, rank mujer: 3)
  Hombre 3  —  Mujer 5   (rank hombre: 5, rank mujer: 2)
  Hombre 4  —  Mujer 6   (rank hombre: 1, rank mujer: 1)
  Hombre 5  —  Mujer 2   (rank hombre: 5, rank mujer: 1)
  Hombre 6  —  Mujer 1   (rank hombre: 1, rank mujer: 1)

Solución 3:
  Hombre 1  —  Mujer 4   (rank hombre: 2, rank mujer: 3)
  Hombre 2  —  Mujer 3   (rank hombre: 5, rank mujer: 3)
  Hombre 3  —  Mujer 2   (rank hombre: 4, rank mujer: 4)
  Hombre 4  —  Mujer 6   (rank hombre: 1, rank mujer: 1)
  Hombre 5  —  Mujer 5   (rank hombre: 4, rank muj

In [34]:
parejas = printer.get_results()

parejas

[[(0, 3), (1, 4), (2, 1), (3, 5), (4, 2), (5, 0)],
 [(0, 3), (1, 2), (2, 4), (3, 5), (4, 1), (5, 0)],
 [(0, 3), (1, 2), (2, 1), (3, 5), (4, 4), (5, 0)],
 [(0, 2), (1, 3), (2, 1), (3, 5), (4, 4), (5, 0)],
 [(0, 2), (1, 3), (2, 4), (3, 5), (4, 1), (5, 0)]]

In [ ]:
possible_parejas = [(0,3), (1,4), (2,1), (3,5), (4,2), (5,0) ]



def validar(possible_parejas):
    for m, w in possible_parejas:
        for m0, w0 in possible_parejas:
            if tablero_m[m][w0] < tablero_m[m][w] and tablero_w[w0][m] < tablero_w[w0][m0]:
                print(f"Pareja inestable: Hombre {m} con Mujer {w0} y Hombre {m0} con Mujer {w}")
                return "INVALID"
                
            

    return "VALID"

resultado = validar(possible_parejas)

resultado

'VALID'

In [35]:
validaciones = list(map(validar, printer.get_results()))

validaciones


['VALID', 'VALID', 'VALID', 'VALID', 'VALID']

In [33]:
def tablero(pareja):
    table_m = [[0]*6 for _ in range(6)]
    table_w = [[0]*6 for _ in range(6)]

    for i in range(6):
        for j in range(6):
            if pareja[i][1] == j:
                table_m[i][j] = tablero_m[i][j]
                table_w[j][i] = tablero_w[j][i]

    return table_m, table_w

In [38]:
solucion_m, solucion_w = tablero(parejas[0])

In [39]:
solucion_m, solucion_w

([[0, 0, 0, 2, 0, 0],
  [0, 0, 0, 0, 4, 0],
  [0, 4, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 1],
  [0, 0, 3, 0, 0, 0],
  [1, 0, 0, 0, 0, 0]],
 [[0, 0, 0, 0, 0, 1],
  [0, 0, 4, 0, 0, 0],
  [0, 0, 0, 0, 4, 0],
  [3, 0, 0, 0, 0, 0],
  [0, 4, 0, 0, 0, 0],
  [0, 0, 0, 1, 0, 0]])